<a href="https://colab.research.google.com/github/MitjaGo/YT/blob/main/MP3_editor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# --- STEP 0: Install required packages ---
!pip install --quiet numpy==2.3.4 pydub mutagen shazamio requests nest_asyncio
!apt-get install -y ffmpeg

# --- STEP 1: Imports and setup ---
import warnings, os, re, shutil, requests, asyncio, nest_asyncio
from google.colab import files
from pydub import AudioSegment, silence
from mutagen.easyid3 import EasyID3
from shazamio import Shazam

warnings.filterwarnings("ignore")
nest_asyncio.apply()  # Fix for Colab's running event loop

# --- STEP 2: Upload MP3 file ---
uploaded = files.upload()
for fn in uploaded.keys():
    mp3_file = fn
    print(f"Uploaded file: {fn}")

# --- STEP 3: Read ID3 tags ---
audio = EasyID3(mp3_file)
default_title = audio.get('title', ['Unknown Title'])[0]
default_artist = audio.get('artist', ['Unknown Artist'])[0]

print(f"Original Title: {default_title}")
print(f"Original Artist: {default_artist}")

# --- STEP 4: Split audio by silence ---
audio_seg = AudioSegment.from_file(mp3_file)
mean_db = audio_seg.dBFS
sil_thresh = mean_db - 16
print(f"Auto-detected silence threshold: {sil_thresh:.2f} dBFS")

chunks = silence.split_on_silence(
    audio_seg,
    min_silence_len=600,
    silence_thresh=sil_thresh
)

temp_folder = "temp_splits"
os.makedirs(temp_folder, exist_ok=True)
temp_files = []

for i, chunk in enumerate(chunks):
    chunk = AudioSegment.silent(duration=100) + chunk + AudioSegment.silent(duration=100)
    temp_filename = os.path.join(temp_folder, f"chunk_{i+1}.mp3")
    chunk.export(temp_filename, format="mp3")
    temp_files.append(temp_filename)

print(f"Created {len(temp_files)} audio chunks.")

# --- STEP 5: Recognize chunks with Shazam ---
shazam = Shazam()

async def recognize_chunks(files):
    chunk_info = []
    for file in files:
        try:
            out = await shazam.recognize_song(file)
            track_title = out['track']['title']
            track_artist = out['track']['subtitle']
            track_image = out['track']['images']['coverart'] if 'images' in out['track'] else None
            chunk_info.append((file, track_title, track_artist, track_image))
        except Exception:
            chunk_info.append((file, None, None, None))
    return chunk_info

loop = asyncio.get_event_loop()
chunk_info = loop.run_until_complete(recognize_chunks(temp_files))

# --- STEP 6: Merge consecutive chunks of the same song ---
def sanitize_filename(name):
    return "".join(c if c.isalnum() or c in " _-." else "_" for c in name)

merged_chunks = []
current_chunk = None

for info in chunk_info:
    file, title, artist, img = info
    if title is None:
        title = default_title
        artist = default_artist

    title_safe = sanitize_filename(title)
    artist_safe = sanitize_filename(artist)

    if current_chunk is None:
        current_chunk = {'title': title_safe, 'artist': artist_safe, 'image': img, 'audio_files':[file]}
    else:
        if title_safe == current_chunk['title'] and artist_safe == current_chunk['artist']:
            current_chunk['audio_files'].append(file)
        else:
            merged_chunks.append(current_chunk)
            current_chunk = {'title': title_safe, 'artist': artist_safe, 'image': img, 'audio_files':[file]}

if current_chunk:
    merged_chunks.append(current_chunk)

print(f"Merged into {len(merged_chunks)} final chunks.")

# --- STEP 7: Export merged MP3s with ID3 tags and thumbnails ---
output_folder = "splits"
os.makedirs(output_folder, exist_ok=True)
final_files = []

for idx, chunk in enumerate(merged_chunks, start=1):
    # Merge audio files in this chunk
    merged_audio = AudioSegment.empty()
    for f in chunk['audio_files']:
        merged_audio += AudioSegment.from_file(f)

    # Safe filename
    split_filename = os.path.join(output_folder, f"{idx:02d}_{chunk['title']}.mp3")
    merged_audio.export(split_filename, format="mp3")
    final_files.append(split_filename)

    # Set ID3 tags
    audio_id3 = EasyID3(split_filename)
    audio_id3['title'] = chunk['title']
    audio_id3['artist'] = chunk['artist']
    audio_id3.save()

    # Download thumbnail
    if chunk['image']:
        img_data = requests.get(chunk['image']).content
        img_filename = os.path.join(output_folder, f"{idx:02d}_{chunk['title']}.jpg")
        with open(img_filename, 'wb') as f:
            f.write(img_data)

print(f"✅ Exported {len(final_files)} merged chunks with ID3 tags and thumbnails.")

# --- STEP 8: Zip all merged chunks for download ---
zip_filename = "mp3_chunks_merged.zip"
shutil.make_archive("mp3_chunks_merged", 'zip', output_folder)
files.download(zip_filename)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


Saving Le Migliori Canzoni Di Zucchero.mp3 to Le Migliori Canzoni Di Zucchero (1).mp3
Uploaded file: Le Migliori Canzoni Di Zucchero (1).mp3
Original Title: Le Migliori Canzoni Di Zucchero - Zucchero Album Completo - Best Of Zucchero
Original Artist: Italia radio
Auto-detected silence threshold: -28.45 dBFS
Created 111 audio chunks.


/tmp/ipython-input-3063202864.py:60: DeprecationWarning: Call to deprecated function recognize_song. Use recognize method instead of recognize_song
  out = await shazam.recognize_song(file)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/tmp/ipython-input-3063202864.py:60: DeprecationWarning: Call to deprecated function recognize_song. Use recognize method instead of recognize_song
  out = await shazam.recognize_song(file)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow(

Merged into 58 final chunks.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


✅ Exported 58 merged chunks with ID3 tags and thumbnails.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

fade out

In [ ]:
# Install dependencies
!pip install pydub
!apt-get install ffmpeg -y

# Import libraries
from google.colab import files
from pydub import AudioSegment
import zipfile
import os

# Step 1: Upload ZIP of MP3s
uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith(".zip"):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall("mp3_files")

os.makedirs("processed_mp3s", exist_ok=True)

# Step 2: Apply fade out (last 5%)
input_dir = "mp3_files"
output_dir = "processed_mp3s"

for file in os.listdir(input_dir):
    if file.lower().endswith(".mp3"):
        audio_path = os.path.join(input_dir, file)
        audio = AudioSegment.from_mp3(audio_path)

        # Fade out last 5% of the track
        fade_duration_ms = int(len(audio) * 0.05)
        faded_audio = audio.fade_out(fade_duration_ms)

        # Export processed file
        faded_audio.export(os.path.join(output_dir, file), format="mp3")

print("All files processed and saved in 'processed_mp3s'")

# Step 3: Zip processed files for download
!zip -r processed_mp3s.zip processed_mp3s
files.download("processed_mp3s.zip")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
